In [32]:
import json
import subprocess
from pathlib import Path

import numpy as np
from xopen import xopen

# raw

In [3]:
PALOMA_DIR = "../scratch/paloma_c4_100_domains_val"

In [8]:
paloma_files = list(Path(PALOMA_DIR).iterdir())

In [10]:
obj_list = []
with xopen(paloma_files[0], "r") as in_file:
    for line in in_file:
        obj = json.loads(line)
        obj_list.append(obj)

In [17]:
obj = obj_list[0]

In [18]:
obj.keys()

dict_keys(['text', 'id', 'added', 'source', 'subdomain'])

In [19]:
obj["text"]

'What does the future hold for Star Wars animation?\nIt\'s a sad day for fans of Star Wars: The Clone Wars... from a certain point of view.\nLucasfilm and The Clone Wars\' supervising director, Dave Filoni, have announced that Star Wars: The Clone Wars, the television series, is over and that the fifth season that just came to an end was the final one for the show.\nBut here\'s where it gets complicated. More episodes of The Clone Wars have been made than what we\'ve seen. There were in fact glimpses in the trailer for Season 5 of a storyline that ended up being held, and there have been other references made by Filoni and others, as far back as Star Wars Celebration VI last summer, indicating more stories were being produced for a presumed sixth season. In fact, when I recently spoke to Filoni for a Post-Season 5 interview, he told me, "There are more stories to tell, and as I’ve said, we’ve been working on them for awhile."\nThere won\'t be a sixth season, but the announcement did pr

In [20]:
obj["id"]

'bb93185a-cdc7-42a6-8195-df9ebeafd360'

In [21]:
obj["added"]

'2023-04-01T00:16:27.393489+00:00'

In [22]:
obj["source"]

'c4_100_domains'

In [23]:
obj["subdomain"]

'100_www.ign.com'

# tokenized

In [25]:
PALOMA_BIN = "../scratch/paloma_c4_100_domains_val_tokenized.bin"

In [26]:
data = np.memmap(PALOMA_BIN, dtype=np.uint16, mode="r")

In [29]:
f"{len(data):_}"

'9_761_402'

# replicate

In [35]:
cmd = (
    f"python -m lb.scripts.extract_paloma "
    f"--paloma_dir ../scratch/paloma_c4_100_domains_val "
    f"--text_dir ../scratch/text/paloma"
)
subprocess.run(cmd, shell=True)

paloma: 100%|████████████████████████| 99/99 [00:02<00:00, 40.56it/s]


CompletedProcess(args='python -m lb.scripts.extract_paloma --paloma_dir ../scratch/paloma_c4_100_domains_val --text_dir ../scratch/text/paloma', returncode=0)

In [54]:
cmd = (
    f"python -m lb.scripts.tokenize "
    f"--text_dir ../scratch/text/paloma "
    f"--tokens_file ../scratch/tokenized/paloma.bin"
)
subprocess.run(cmd, shell=True)

text_files: 100%|█████████████| 14059/14059 [00:32<00:00, 429.68it/s]


CompletedProcess(args='python -m lb.scripts.tokenize --text_dir ../scratch/text/paloma --tokens_file ../scratch/tokenized/paloma.bin', returncode=0)

In [61]:
arr_want = np.memmap(
    "../scratch/paloma_c4_100_domains_val_tokenized.bin", dtype=np.uint16, mode="r"
)

In [62]:
arr_have = np.memmap("../scratch/tokenized/paloma.bin", dtype=np.uint16, mode="r")

In [63]:
len(arr_want)

9761402

In [64]:
len(arr_have)

9761402

In [65]:
arr_want

memmap([ 3666,  3656,   290, ...,   767,    13, 50256], dtype=uint16)

In [66]:
arr_have

memmap([15905,   447,   247, ...,  4621,    13, 50256], dtype=uint16)

In [68]:
arr_want.sum()

44563429315

In [69]:
arr_have.sum()

44563429315

^ equivalent upto ordering of documents